In [75]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [76]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,count,max,min,avg,udf
from pyspark.sql.types import DoubleType,IntegerType
spark=SparkSession.builder.appName("project2").getOrCreate()

In [77]:
df=spark.read.csv("/content/drive/MyDrive/ColabNotebooks/project2/OfficeDataProject.csv",inferSchema=True,header=True)
df.show(1)

+-----------+-------------+----------+-----+------+---+-----+
|employee_id|employee_name|department|state|salary|age|bonus|
+-----------+-------------+----------+-----+------+---+-----+
|       1000|    Nitz Leif| Marketing|   CA|  6131| 26|  543|
+-----------+-------------+----------+-----+------+---+-----+
only showing top 1 row


In [78]:
print("Total Number of employees=",df.count())

Total Number of employees= 1000


In [79]:
print("Total number of departments=",df.select(col("department")).distinct().count())

Total number of departments= 6


In [80]:
df.select(col("department")).distinct().show()

+----------+
|department|
+----------+
|     Sales|
|        HR|
|   Finance|
|Purchasing|
| Marketing|
|  Accounts|
+----------+



In [81]:
df.groupBy("department").agg(count("*").alias("Total employees")).show()

+----------+---------------+
|department|Total employees|
+----------+---------------+
|     Sales|            169|
|        HR|            171|
|   Finance|            162|
|Purchasing|            166|
| Marketing|            170|
|  Accounts|            162|
+----------+---------------+



In [82]:
df.groupBy("state").agg(count("*").alias("Total employees")).show()

+-----+---------------+
|state|Total employees|
+-----+---------------+
|   LA|            205|
|   CA|            205|
|   WA|            208|
|   NY|            173|
|   AK|            209|
+-----+---------------+



In [83]:
df.groupBy("state","department").agg(count("*").alias("Total employees")).show()

+-----+----------+---------------+
|state|department|Total employees|
+-----+----------+---------------+
|   CA|     Sales|             42|
|   CA| Marketing|             33|
|   NY|  Accounts|             34|
|   NY|     Sales|             27|
|   CA|   Finance|             35|
|   CA|  Accounts|             35|
|   CA|Purchasing|             32|
|   WA|        HR|             47|
|   AK|Purchasing|             30|
|   WA|  Accounts|             27|
|   WA|Purchasing|             38|
|   AK|     Sales|             38|
|   AK|  Accounts|             37|
|   WA| Marketing|             39|
|   LA|        HR|             41|
|   LA|     Sales|             35|
|   AK|        HR|             25|
|   LA|   Finance|             29|
|   AK|   Finance|             37|
|   LA|Purchasing|             45|
+-----+----------+---------------+
only showing top 20 rows


In [84]:
df.groupBy("department").agg(min("salary").alias("Minimum Salary"),max("salary").alias("Maximum Salary")).orderBy(col("Minimum Salary").asc(),col("Maximum Salary").asc()).show()

+----------+--------------+--------------+
|department|Minimum Salary|Maximum Salary|
+----------+--------------+--------------+
|   Finance|          1006|          9899|
|  Accounts|          1007|          9890|
|        HR|          1013|          9982|
| Marketing|          1031|          9974|
|     Sales|          1103|          9982|
|Purchasing|          1105|          9985|
+----------+--------------+--------------+



In [85]:
average=df.filter(df.state=="NY").groupBy(df.state).agg(avg("bonus").alias("avg_bonus")).select("avg_bonus").collect()[0]["avg_bonus"]
df.filter((df.state=="NY") & (df.department=="Finance")&(df.bonus>average)).show()

+-----------+--------------------+----------+-----+------+---+-----+
|employee_id|       employee_name|department|state|salary|age|bonus|
+-----------+--------------------+----------+-----+------+---+-----+
|       1035|       Vivan Sifford|   Finance|   NY|  1129| 35| 1261|
|       1073|      Herder Gallman|   Finance|   NY|  1988| 31| 1402|
|       1082|          Nena Rocha|   Finance|   NY|  3417| 25| 1647|
|       1087|       Leif Lemaster|   Finance|   NY|  8642| 45| 1782|
|       1100|Ellingsworth Meli...|   Finance|   NY|  7845| 32| 1358|
|       1127|        Escoto Gilma|   Finance|   NY|  3426| 41| 1285|
|       1161|     Georgeanna Laub|   Finance|   NY|  2469| 26| 1679|
|       1175|     Durio Tenenbaum|   Finance|   NY|  2253| 42| 1684|
|       1180|       Juliana Grigg|   Finance|   NY|  8178| 42| 1617|
|       1215|        Tiffani Benz|   Finance|   NY|  1665| 41| 1969|
|       1220|          Nitz Ilana|   Finance|   NY|  2443| 50| 1342|
|       1342|   Phylicia Antonina|

In [88]:
def total_salary(age,salary):
  if age>45:
    sum=salary + 500
    return sum
  else:
    return salary

salary_increment=udf(lambda x,y:total_salary(x,y),IntegerType())
df=df.withColumn("salary",salary_increment(col("age"),col("salary")))

In [90]:

df.filter(col("age")>45).write.csv("/content/drive/MyDrive/ColabNotebooks/project2/exports/TranformedData.csv")